In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import os,errno
import sys
import cartopy.feature as cfeature
import matplotlib.ticker as mticker
import pandas as pd
import datetime as dt
from scipy.ndimage.measurements import label
from math import sin, cos, sqrt, atan2, radians
import geopy.distance
%matplotlib inline

dir2='/thorncroftlab_rit/ahenny/rain/'
dir1='/thorncroftlab_rit/ahenny/rain/US/ghcnd_all/'
dir='/thorncroftlab_rit/ahenny/rain/DISSERTATION_SCRIPTS_RESULTS/'

In [ ]:
#Taiwan bounds: 0-50 N,80-160 E
#NEUSA bounds: 10-60 N,30-115 W
#These are large enough that an AR invading the NEUSA or Taiwan would be detected - i.e. >2000 km on all sides
#ds=xr.open_dataset(dir+'ls_extreme_rain_taiwan.nc')
#dates=ds.large_scale_extreme_days

#dz2=xr.open_dataset(dir+'ivt_basics_neusa_90.nc')
#longitude_sub=dz2.longitude.values
ds2=xr.open_dataset(dir+'ivt_basics_neusa.nc')#change this threshold for sensitivity testing #_95_summer
#ds2['longitude']=longitude_sub

#Winter  425.82147
#Spring 448.2696
#Summer 534.098
#Fall 528.6943
#warm 531.6989

ivt_threshold=531.6989

ivt_mag=ds2['ivt_mag']
ivtx=ds2['ivtx']
ivty=ds2['ivty']
lons_full=ivt_mag.longitude.values
lats_full=ivt_mag.latitude.values
print(ivt_mag)

lat_range=np.arange(0,60,0.25)
lon_range=np.arange(-140,-110,0.25)

if 1==0:
    ds3=xr.open_dataset(dir+'era_5_fall_u_850_neusa.nc')
    u_850=ds3['u']#.sel(latitude=lat_range,longitude=lon_range)
    ds4=xr.open_dataset(dir+'era_5_fall_v_850_neusa.nc')
    v_850=ds4['v']#.sel(latitude=lat_range,longitude=lon_range)
    ds5=xr.open_dataset(dir+'era_5_fall_mslp_neusa.nc')
    print(ds5)
    slp=ds5['msl']#.sel(latitude=lat_range,longitude=lon_range)/100.

    ds6=xr.open_dataset(dir+'era_5_fall_sfc_remnants_neusa.nc')
    ds6['longitude']=ds3.longitude.values
    print(ds6)
    slp_2=ds6['mslp']#.sel(latitude=lat_range,longitude=lon_range)
    ds7=xr.open_dataset(dir+'era_5_fall_pl_remnants_neusa.nc')
    ds7['longitude']=ds3.longitude.values
    u_850_2=ds7['u']#.sel(latitude=lat_range,longitude=lon_range)
    v_850_2=ds7['v']#.sel(latitude=lat_range,longitude=lon_range)

In [ ]:
print(u_850)

In [ ]:
ds=xr.open_dataset(dir+'extreme_days_ghcnd_99station_80area95.nc')
lats=ds['lats'].values.tolist()
lons=ds['lons'].values.tolist()
lons=[x+360. for x in lons]
dates=ds['dates'].values
dates_unique=list(set(dates))
dates_unique=pd.DatetimeIndex(dates_unique).sort_values()
stations=ds['stations'].values.tolist()
obs=ds['obs'].values.tolist()
print(dates)

ds1=xr.open_dataset(dir+'station_numbers_95.nc')
lats1=ds1['lats']#use these lists for plotting
lons1=ds1['lons']+360.
stations1=ds1['stations']
thresholds=ds1['thresholds']/10.

In [ ]:
from copy import copy
import shapely.geometry as sgeom
def find_side(ls, side):
    """
    Given a shapely LineString which is assumed to be rectangular, return the
    line corresponding to a given side of the rectangle.
    
    """
    minx, miny, maxx, maxy = ls.bounds
    points = {'left': [(minx, miny), (minx, maxy)],
              'right': [(maxx, miny), (maxx, maxy)],
              'bottom': [(minx, miny), (maxx, miny)],
              'top': [(minx, maxy), (maxx, maxy)],}
    return sgeom.LineString(points[side])


def lambert_xticks(ax, ticks):
    """Draw ticks on the bottom x-axis of a Lambert Conformal projection."""
    te = lambda xy: xy[0]
    lc = lambda t, n, b: np.vstack((np.zeros(n) + t, np.linspace(b[2], b[3], n))).T
    xticks, xticklabels = _lambert_ticks(ax, ticks, 'bottom', lc, te)
    ax.xaxis.tick_bottom()
    ax.set_xticks(xticks)
    ax.set_xticklabels([ax.xaxis.get_major_formatter()(xtick) for xtick in xticklabels])
    

def lambert_yticks(ax, ticks):
    """Draw ricks on the left y-axis of a Lamber Conformal projection."""
    te = lambda xy: xy[1]
    lc = lambda t, n, b: np.vstack((np.linspace(b[0], b[1], n), np.zeros(n) + t)).T
    yticks, yticklabels = _lambert_ticks(ax, ticks, 'left', lc, te)
    ax.yaxis.tick_left()
    ax.set_yticks(yticks)
    ax.set_yticklabels([ax.yaxis.get_major_formatter()(ytick) for ytick in yticklabels])
def _lambert_ticks(ax, ticks, tick_location, line_constructor, tick_extractor):
    """Get the tick locations and labels for an axis of a Lambert Conformal projection."""
    outline_patch = sgeom.LineString(ax.outline_patch.get_path().vertices.tolist())
    axis = find_side(outline_patch, tick_location)
    n_steps = 30
    extent = ax.get_extent(ccrs.PlateCarree())
    _ticks = []
    for t in ticks:
        xy = line_constructor(t, n_steps, extent)
        proj_xyz = ax.projection.transform_points(ccrs.Geodetic(), xy[:, 0], xy[:, 1])
        xyt = proj_xyz[..., :2]
        ls = sgeom.LineString(xyt.tolist())
        locs = axis.intersection(ls)
        if not locs:
            tick = [None]
        else:
            tick = tick_extractor(locs.xy)
        _ticks.append(tick[0])
    # Remove ticks that aren't visible:    
    ticklabels = copy(ticks)
    while True:
        try:
            index = _ticks.index(None)
        except ValueError:
            break
        _ticks.pop(index)
        ticklabels.pop(index)
    return _ticks, ticklabels

In [ ]:
ds6=xr.open_dataset(dir+'IBTrACS.NA.v04r00.nc')
#print(ds)
lat_tc=ds6.lat
lon_tc=ds6.lon
#nature_tc=ds6.nature
ds7=xr.open_dataset(dir+'ibtracs_na_time.nc')
years_tc=ds7.years
months_tc=ds7.months
days_tc=ds7.days
hours_tc=ds7.hours
nature_tc=ds7.nature#1 if tropical, 0 if not
print(nature_tc)
print(hours_tc)

In [ ]:
years=np.arange(1979,2020,1)
season='fall'
print(years)
if season=='winter':
    for i in range(1,len(years)):
        year=years[i]

        if year%4==0:
            feb_length=29
        else:
            feb_length=28
        base_date_1=dt.datetime(years[i]-1,12,1,6)
        dates_list_1=[base_date_1+dt.timedelta(days=x) for x in range(31)]

        base_date_2=dt.datetime(years[i],1,1,6)
        dates_list_2=[base_date_2+dt.timedelta(days=x) for x in range(31+feb_length)]

        dates_list=dates_list_1+dates_list_2

        if i==1:
            dates_concat=dates_list
        else:
            dates_concat=dates_concat+dates_list
if season=='spring':
    for i in range(len(years)):
        year=years[i]
        base_date=dt.datetime(year,3,1,6)
        dates_list=[base_date+dt.timedelta(days=x) for x in range(31+30+31)]
        if i==0:
            dates_concat=dates_list
        else:
            dates_concat=dates_concat+dates_list
if season=='summer':
    for i in range(len(years)):
        year=years[i]
        base_date=dt.datetime(year,6,1,6)
        dates_list=[base_date+dt.timedelta(days=x) for x in range(30+31+31)]
        if i==0:
            dates_concat=dates_list
        else:
            dates_concat=dates_concat+dates_list
if season=='fall':
    for i in range(len(years)):
        year=years[i]
        base_date=dt.datetime(year,9,1,6)
        dates_list=[base_date+dt.timedelta(days=x) for x in range(30+31+30)]
        if i==0:
            dates_concat=dates_list
        else:
            dates_concat=dates_concat+dates_list
print(dates_concat)
dates_unique=dates_concat
#dates_unique=[x for x in dates_unique if x.year==1979]

In [ ]:
dates_unique=[]
for i in range(10):
    year=2000+i
    date_start=dt.datetime(year,9,1,6)
    date_range=[date_start+dt.timedelta(days=x) for x in range(91)]
    if i==0:
        dates_unique=date_range
    else:
        dates_unique=dates_unique+date_range
print(dates_unique)

In [ ]:
#First - make dataset containing AR points on all four time steps during these 128 EP days
#dates_unique1=dates_unique[0:2]
for d in range(len(dates_unique)):
#for d in range(2):
    date=dates_unique[d]#date is at 6Z automatically.  IVT goes from 6Z on day to 6Z on next day
    date_range=[date+dt.timedelta(hours=6*x) for x in range(4)]
    date_range=pd.DatetimeIndex(date_range).values#date range for IVT selection
    for date in date_range:
        print(date)
        date=pd.to_datetime(date)
        year=date.year
        month=date.month
        day=date.day
        hour=date.hour
        ivt_mag_current=ivt_mag.sel(time=date)
        ivt_extreme=ivt_mag_current.where(ivt_mag_current>=ivt_threshold)
        ivt_extreme_full=ivt_extreme
        ivt_ones=ivt_extreme/ivt_extreme
        ivt_ones=ivt_ones.fillna(0)
        structure = np.ones((3, 3), dtype=np.int)          
        labeled_full,ncomponents_full=label(ivt_ones,structure) 
        labeled_xr_full=xr.zeros_like(ivt_extreme)
        labeled_xr_full[:,:]=labeled_full
        if d==0 and date==date_range[0]:
            labeled_ivt_points=labeled_xr_full
        else:
            labeled_ivt_points=xr.concat([labeled_ivt_points,labeled_xr_full],dim='time')
            
lats_tc_list=lat_tc.values.ravel()
lons_tc_list=lon_tc.values.ravel()
years_tc_list=years_tc.values.ravel()
months_tc_list=months_tc.values.ravel()
days_tc_list=days_tc.values.ravel()
hours_tc_list=hours_tc.values.ravel()
nature_tc_list=nature_tc.values.ravel()

zipped_tc=list(zip(lats_tc_list,lons_tc_list,years_tc_list,months_tc_list,days_tc_list,hours_tc_list,nature_tc_list))
nonzero_tc=[x for x in zipped_tc if x[0]>=0]

lat_array=xr.zeros_like(ivt_mag[0,:,:])
lon_array=xr.zeros_like(ivt_mag[0,:,:])
for i in range(ivt_mag.longitude.size):
    lat_array[:,i]=ivt_mag.latitude.values
for i in range(ivt_mag.latitude.size):
    lon_array[i,:]=ivt_mag.longitude.values
    
lats_full=ivt_mag.latitude.values
lons_full=ivt_mag.longitude.values
lon_array_list=lon_array.values.ravel()
lat_array_list=lat_array.values.ravel()

lats_xr=lat_array.copy()[0::2,0]
lons_xr=lon_array.copy()[0,0::2]
lats_array_xr=lat_array.copy()[0::2,0::2]
lons_array_xr=lon_array.copy()[0::2,0::2]
lats_xr_flat=lats_array_xr.values.ravel()
lons_xr_flat=lons_array_xr.values.ravel()

    
zipped_latlon=list(zip(lat_array_list,lon_array_list))

for d in range(len(dates_unique)):
#for d in range(2):
    print(d)
    date=dates_unique[d]#date is at 6Z automatically.  IVT goes from 6Z on day to 6Z on next day
    date_range=[date+dt.timedelta(hours=6*x) for x in range(4)]
    date_range=pd.DatetimeIndex(date_range).values#date range for IVT selection
    for date in date_range:
        print(date)
        date=pd.to_datetime(date)
        year=date.year
        month=date.month
        day=date.day
        hour=date.hour
        ivt_mag_current=ivt_mag.sel(time=date)
        ivtx_current=ivtx.sel(time=date)
        ivty_current=ivty.sel(time=date)

        ivt_extreme=ivt_mag_current.where(ivt_mag_current>=ivt_threshold)
        ivt_extreme_full=ivt_extreme
        ivt_ones=ivt_extreme/ivt_extreme
        ivt_ones=ivt_ones.fillna(0)
        
        ###Now load TC data and find regions within a certain radius (currently 500 km) of TC centers
        
        check_lat_tc_trop_list=[x[0] for x in nonzero_tc if x[2]==year and x[3]==month and x[4]==day and x[5]==hour and x[6]==1]
        check_lat_tc_nontrop_list=[x[0] for x in nonzero_tc if x[2]==year and x[3]==month and x[4]==day and x[5]==hour and x[6]==0]
        check_lon_tc_trop_list=[x[1] for x in nonzero_tc if x[2]==year and x[3]==month and x[4]==day and x[5]==hour and x[6]==1]
        check_lon_tc_nontrop_list=[x[1] for x in nonzero_tc if x[2]==year and x[3]==month and x[4]==day and x[5]==hour and x[6]==0]

        #print('A')
        
        if len(check_lat_tc_trop_list)>0 and len(check_lon_tc_trop_list)>0:
            #print('TROP')
            for i in range(len(check_lat_tc_trop_list)):
                lat_tc_single=check_lat_tc_trop_list[i]
                lon_tc_single=check_lon_tc_trop_list[i]
                first_elim=[x for x in zipped_latlon if abs(x[0]-lat_tc_single)<5.0 or abs(x[1]-lon_tc_single)<10.0]
                select_ball_trop=[x for x in first_elim if geopy.distance.distance((x[0],x[1]),(lat_tc_single,lon_tc_single)).km<500.]    
                if i==0:
                    all_trop_points=select_ball_trop
                else:
                    all_trop_points=list(set(all_trop_points+select_ball_trop))
            all_trop_pairs=[(x[0],x[1]) for x in all_trop_points]
                    
        if len(check_lat_tc_nontrop_list)>0 and len(check_lon_tc_nontrop_list)>0:
            #print('NONTROP')
            for i in range(len(check_lat_tc_nontrop_list)):
                lat_tc_single=check_lat_tc_nontrop_list[i]
                lon_tc_single=check_lon_tc_nontrop_list[i]
                first_elim=[x for x in zipped_latlon if abs(x[0]-lat_tc_single)<5.0 or abs(x[1]-lon_tc_single)<10.0]
                select_ball_nontrop=[x for x in first_elim if geopy.distance.distance((x[0],x[1]),(lat_tc_single,lon_tc_single)).km<500.]
                if i==0:
                    all_nontrop_points=select_ball_nontrop
                else:
                    all_nontrop_points=list(set(all_nontrop_points+select_ball_nontrop))
            all_nontrop_pairs=[(x[0],x[1]) for x in all_nontrop_points]
        
        labeled_tc_trop=xr.zeros_like(ivt_mag_current)
        labeled_tc_nontrop=xr.zeros_like(ivt_mag_current)
        for i in range(len(lats_full)):
            for j in range(len(lons_full)):
                if len(check_lat_tc_trop_list)>0:
                    if (lats_full[i],lons_full[j]) in all_trop_pairs:
                        labeled_tc_trop[i,j]=1
                if len(check_lat_tc_nontrop_list)>0:
                    if (lats_full[i],lons_full[j]) in all_nontrop_pairs:
                        labeled_tc_nontrop[i,j]=1
                    
        #print('B')
        #print(labeled_tc_trop.max().values)
        
        structure = np.ones((3, 3), dtype=np.int)          
        ivt_ones_ar=ivt_ones.where(labeled_tc_trop==0).fillna(0)
        labeled,ncomponents=label(ivt_ones_ar,structure)

        #AR length is greatest distance between two points in feature
        labeled_xr=xr.zeros_like(ivt_extreme)
        labeled_xr[:,:]=labeled

        endpoint1_list=[]
        endpoint2_list=[]
        distance_max_list=[]
        mean_width_list=[]
        north_component_list=[]
        ivt_mag_current=ivt_mag_current[0::2,0::2]
        ivtx_current=ivtx_current[0::2,0::2]
        ivty_current=ivty_current[0::2,0::2]
        labeled_xr_full=labeled_xr
        labeled_xr=labeled_xr[0::2,0::2]
        labeled_xr_flat=labeled_xr.values.ravel()
        
        #print('C')
        
        nums_list=[]
        points_boundary_list=[]
        lat_first_list=[]
        lon_first_list=[]
        boundaries_xr=xr.zeros_like(labeled_xr)
        area_sum_list=[]

        for i in range(ncomponents+1):
            select_component=labeled_xr.where(labeled_xr==i)
            select_component=select_component-select_component
            select_component=select_component+1
            num_connected=select_component.sum().values
            nums_list.append(num_connected)
            
        zipped_num=list(zip(np.arange(ncomponents+1),nums_list))
        sorted_num=sorted(zipped_num,key=lambda x:x[1])
        largest_num=sorted_num[-1][0]
        zero_indices=[x[0] for x in sorted_num if x[1]==0]
        components_num_list=[x for x in np.arange(ncomponents+1) if x!=largest_num and x not in zero_indices]
        
        ivt_extreme=ivt_extreme[0::2,0::2]
        
        ar_timestep=xr.zeros_like(labeled_xr_full)
        artc_timestep=xr.zeros_like(labeled_xr_full)
        
        labeled_full_select=labeled_ivt_points.sel(time=date)
        
        for i in components_num_list:
            labeled_test=labeled_xr.where(labeled_xr==i)
            set_number_subset=labeled_full_select.where(labeled_xr_full==i)
            set_number_values=set_number_subset.values.flatten().tolist()
            set_number_values=[x for x in set_number_values if x>0]
            set_number_value=list(set(set_number_values))
            set_number_value=set_number_value[0]
            corresponding_full_label=labeled_full_select.where(labeled_full_select==set_number_value)
            corresponding_full_label=corresponding_full_label/corresponding_full_label
            
            distance_max=0
            labeled_select=labeled_xr.where(labeled_xr==i)
            labeled_select=labeled_select/labeled_select
            labeled_sum=labeled_select.sum().values
            
            labeled_full_notc=labeled_full_select.where(labeled_tc_trop==0).fillna(0)
            
            lats_where=lats_xr.where(labeled_xr==i)
            lons_where=lons_xr.where(labeled_xr==i)
            min_lat=lats_where.min(skipna=True).values
            max_lat=lats_where.max(skipna=True).values
            min_lon=lons_where.min(skipna=True).values
            max_lon=lons_where.max(skipna=True).values

            #print('D')
            if labeled_select.sum(skipna=True).values>0:
                distance_hyp=geopy.distance.geodesic((min_lat,min_lon),(max_lat,max_lon)).km
                if distance_hyp>=2000:#filter out features that could not possibly be long enough
                    proceed='no'
                    ivtx_extreme=ivtx_current.where(labeled_xr==i)
                    ivty_extreme=ivty_current.where(labeled_xr==i)
                    ivtx_mean=ivtx_extreme.mean(dim=('latitude','longitude'),skipna=True).values
                    ivty_mean=ivty_extreme.mean(dim=('latitude','longitude'),skipna=True).values

                    north_component_list.append(ivty_mean)
                    area_sum=0
                    #print('E')
                    
                    zipped_label=list(zip(lats_xr_flat,lons_xr_flat,labeled_xr_flat))
                    select_component_lat=[x[0] for x in zipped_label if x[2]==i]
                    select_component_lon=[x[1] for x in zipped_label if x[2]==i]
                    areas=[geopy.distance.distance((x,y),(x,y+0.25)).km*geopy.distance.distance((x,y),(x+0.25,y)).km for x,y in zip(select_component_lat,select_component_lon)]
                    area_sum=float(sum(areas))
                    
                    necessary_length=area_sum/1000.
                    #print(necessary_length)
                    min_lat=min(select_component_lat)
                    max_lat=max(select_component_lat)
                    min_lon=min(select_component_lon)
                    max_lon=max(select_component_lon)
                    
                    zipped_coords_1=list(zip(select_component_lat,select_component_lon))
                    
                    min_lat_select=[x for x in zipped_coords_1 if x[0]==min_lat]
                    sorted_lat=sorted(min_lat_select,key=lambda x:x[1])
                    minlatminlon=sorted_lat[0]
                    
                    max_lat_select=[x for x in zipped_coords_1 if x[0]==max_lat]
                    sorted_lat=sorted(max_lat_select,key=lambda x:x[1])
                    maxlatmaxlon=sorted_lat[-1]
                    
                    min_lon_select=[x for x in zipped_coords_1 if x[1]==min_lon]
                    sorted_lon=sorted(min_lon_select,key=lambda x:x[0])
                    minlonminlat=sorted_lon[0]
                    
                    max_lon_select=[x for x in zipped_coords_1 if x[1]==max_lon]
                    sorted_lon=sorted(max_lon_select,key=lambda x:x[0])
                    maxlonmaxlat=sorted_lon[-1]
                    
                    
                    dist_1=geopy.distance.distance((minlatminlon[0],minlatminlon[1]),(maxlatmaxlon[0],maxlatmaxlon[1])).km
                    dist_2=geopy.distance.distance((minlonminlat[0],minlonminlat[1]),(maxlonmaxlat[0],maxlonmaxlat[1])).km
                    
                    #print(dist_1,dist_2)
                    if dist_1>=necessary_length or dist_2>=necessary_length:
                        if dist_1>=2000. or dist_2>=2000.:
                            proceed='yes'
                            #print('YES')
                    
                    else:
                        for j in range(len(select_component_lat)-1):
                            lat_1=select_component_lat[j]
                            lon_1=select_component_lon[j]
                            for k in range(j+1,len(select_component_lat)):
                                lat_2=select_component_lat[k]
                                lon_2=select_component_lon[k]
                                distance=geopy.distance.distance((lat_1,lon_1),(lat_2,lon_2)).km
                                if distance>distance_max:
                                    distance_max=distance
                            
                        #print('F')
                        if distance_max>=necessary_length and distance_max>=2000.:
                            proceed='yes'
                    
                    north_component=ivty_mean
                    if proceed=='no':# or north_component<50:
                        pass
                    else:
                        #print('G')
                        corresponding_full_label_ar=corresponding_full_label.where(labeled_xr_full==i).fillna(0)
                        corresponding_full_label_tc=corresponding_full_label.where(labeled_tc_trop==1).fillna(0)
                        corresponding_full_label_nontc=corresponding_full_label.where(labeled_tc_nontrop==1).fillna(0)
                        corresponding_full_label_tc_flat=corresponding_full_label_tc.values.ravel()
                        corresponding_full_label_nontc_flat=corresponding_full_label_nontc.values.ravel()
                        #first: region of high IVT component that is AR-qualified
                        #next: region of high IVT component that is within TC shield
                        #Can these be disconnected?  No, because there would have to be non-AR extreme IVT between TC and AR; but that would be connected to AR, so would belong to AR
                        
                        zipped_label=list(zip(lat_array_list,lon_array_list,corresponding_full_label_tc_flat,corresponding_full_label_nontc_flat))
                        select_component_lat=[x[0] for x in zipped_label if x[2]==i]
                        select_component_lon=[x[1] for x in zipped_label if x[2]==i]
                        areas=[geopy.distance.distance((x,y),(x,y+0.25)).km*geopy.distance.distance((x,y),(x+0.25,y)).km for x,y in zip(select_component_lat,select_component_lon)]
                        area_sum_tc=float(sum(areas))
                    
                        area_ratio=area_sum_tc/area_sum
                        if area_ratio>=1./3.:
                            labeled_full=labeled_xr_full.where(labeled_xr_full==i).fillna(0)
                            artc_timestep=artc_timestep+labeled_full
                        else:
                            labeled_full=labeled_xr_full.where(labeled_xr_full==i).fillna(0)
                            ar_timestep=ar_timestep+labeled_full
                    
        ar_timestep=ar_timestep.where(ar_timestep>0)
        ar_timestep=ar_timestep/ar_timestep
        
        artc_timestep=artc_timestep.where(artc_timestep>0)
        artc_timestep=artc_timestep/artc_timestep
        #print('H')
        #sys.exit()
        if d==0 and date==date_range[0]:
            ar_points=ar_timestep
            artc_points=artc_timestep
            tc_points_trop=labeled_tc_trop
            tc_points_nontrop=labeled_tc_nontrop
        else:
            ar_points=xr.concat([ar_points,ar_timestep],dim='time')
            artc_points=xr.concat([artc_points,artc_timestep],dim='time')
            tc_points_trop=xr.concat([tc_points_trop,labeled_tc_trop],dim='time')
            tc_points_nontrop=xr.concat([tc_points_nontrop,labeled_tc_nontrop],dim='time')

            
dk=xr.Dataset()

dk['ar']=(('time','latitude','longitude'),ar_points)
dk['artc']=(('time','latitude','longitude'),artc_points)
dk['tc_trop']=(('time','latitude','longitude'),tc_points_trop)
dk['tc_nontrop']=(('time','latitude','longitude'),tc_points_nontrop)
dk['labeled']=(('time','latitude','longitude'),labeled_ivt_points)

dk.coords['time']=ar_points.time
dk.coords['latitude']=ar_points.latitude
dk.coords['longitude']=ar_points.longitude
#A->B, B->C, G->H
try:
    #os.remove(dir+'neusa_ar_points_test_var95const_ALL_2000_2009.nc')
    os.remove(dir+'neusa_ar_points_fallwarm_var95const.nc')#done: 90,925,95,95const
except OSError:
    pass
#dk.to_netcdf(dir+'neusa_ar_points_test_var95const_ALL_2000_2009.nc',mode='w',format='NETCDF4')
dk.to_netcdf(dir+'neusa_ar_points_fallwarm_var95const.nc',mode='w',format='NETCDF4')
#original = _test
#area condition for extended TC moisture flux = _test_elim1